In [1]:
import pandas as pd
import datetime

In [2]:
trips=pd.read_csv("rt_trips_DB_2018.txt",sep=";")

In [3]:
trips.head()

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"


In [4]:
trips.shape

(2182637, 16)

In [5]:
trips.DATASOURCE.describe()

count     2182637
unique          1
top            DB
freq      2182637
Name: DATASOURCE, dtype: object

Datasource is useless

In [6]:
trips.DAYOFSERVICE.describe()

count                2182637
unique                   360
top       12-FEB-18 00:00:00
freq                    7122
Name: DAYOFSERVICE, dtype: object

In [7]:
trips.TRIPID.describe()

count    2.182637e+06
mean     7.149479e+06
std      7.314423e+05
min      5.955221e+06
25%      6.592584e+06
50%      7.110090e+06
75%      8.018241e+06
max      8.592207e+06
Name: TRIPID, dtype: float64

In [8]:
trips.LINEID.describe()

count     2182637
unique        130
top           46A
freq        76728
Name: LINEID, dtype: object

In [9]:
trips.ROUTEID.describe()

count     2182637
unique        588
top        46A_74
freq        37182
Name: ROUTEID, dtype: object

In [10]:
trips.DIRECTION.describe()

count    2.182637e+06
mean     1.504103e+00
std      4.999833e-01
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      2.000000e+00
max      2.000000e+00
Name: DIRECTION, dtype: float64

In [11]:
trips[["PLANNEDTIME_ARR","PLANNEDTIME_DEP","ACTUALTIME_ARR","ACTUALTIME_DEP"]].describe()

,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
count,2.182637e+06,2.182637e+06,2.045430e+06,2.018086e+06
mean,5.507875e+04,5.153329e+04,5.506239e+04,5.170612e+04
std,1.742482e+04,1.757164e+04,1.733870e+04,1.752695e+04
min,1.820300e+04,1.620000e+04,1.754600e+04,1.597400e+04
25%,3.968300e+04,3.600000e+04,3.966800e+04,3.651800e+04
50%,5.527600e+04,5.160000e+04,5.518900e+04,5.161800e+04
75%,6.900600e+04,6.516000e+04,6.901900e+04,6.539800e+04
max,9.168000e+04,8.760000e+04,9.437800e+04,9.717700e+04


Around 5% of actual values are missing. should be able to be retrieved from trips data, so we keep data for now. Also, actual times are float. It's worth to check whether they actually contain decimals or could be stored as ints:

In [14]:
sum(trips["ACTUALTIME_ARR"]%1!=0)

137207

In [16]:
sum(trips["ACTUALTIME_DEP"]%1!=0)

164551

Thus, float should be preserved.

In [13]:
trips.BASIN.describe()

count     2182637
unique          1
top        BasDef
freq      2182637
Name: BASIN, dtype: object

BASIN is useless.

In [12]:
trips[["TENDERLOT","SUPPRESSED","JUSTIFICATIONID"]].describe()

,TENDERLOT,SUPPRESSED,JUSTIFICATIONID
count,0.0,4333.0,4330.000000
mean,NaN,0.0,237554.867667
std,NaN,0.0,34585.051789
min,NaN,0.0,177856.000000
25%,NaN,0.0,205506.500000
50%,NaN,0.0,233851.500000
75%,NaN,0.0,271506.750000
max,NaN,0.0,297899.000000


TENDERLOT, SUPPRESSED, JUSTIFICATIONID seem useless

LASTUPDATE is of no interest

In [15]:
trips.NOTE.describe()

count       2182637
unique        46690
top       ,2422401,
freq            244
Name: NOTE, dtype: object

No clue what this could be for. We dropped it the last time, so i won't keep it now.

Convert DAYOFSERVICE to Unix Timestamp:

In [17]:
trips["DAYSTAMP"]=trips["DAYOFSERVICE"].apply((lambda x:int(datetime.datetime.strptime(x.split(" ")[0], "%d-%b-%y").timestamp())))

In [21]:
trips["DAYSTAMP"].astype("int64",copy=False)

0          1517961600
1          1517961600
2          1517961600
3          1517961600
4          1517961600
              ...    
2182632    1526252400
2182633    1526252400
2182634    1526252400
2182635    1526252400
2182636    1526252400
Name: DAYSTAMP, Length: 2182637, dtype: int64

In [24]:
trips["ROUTE"]=trips["ROUTEID"].apply(lambda x:x.split("_")[1])

In [25]:
trips["ROUTE"].dtype

dtype('O')

In [26]:
trips["ROUTE"].describe()

count     2182637
unique        139
top            15
freq       103223
Name: ROUTE, dtype: object

In [29]:
trips=trips.drop(["DATASOURCE","DAYOFSERVICE","BASIN","TENDERLOT","SUPPRESSED","JUSTIFICATIONID","LASTUPDATE","NOTE","ROUTEID"],axis=1)

In [31]:
trips.head()

,TRIPID,LINEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,DAYSTAMP,ROUTE
0,6253783,68,1,87245,84600,87524.0,84600.0,1517961600,80
1,6262138,25B,2,30517,26460,32752.0,NaN,1517961600,271
2,6254942,45A,2,35512,32100,36329.0,32082.0,1517961600,70
3,6259460,25A,1,57261,54420,58463.0,54443.0,1517961600,273
4,6253175,14,1,85383,81600,84682.0,81608.0,1517961600,15


In [32]:
trips.shape

(2182637, 9)

In [33]:
trips.rename(columns={'ROUTE':'ROUTEID'}, inplace=True)

In [34]:
trips.columns

Index(['TRIPID', 'LINEID', 'DIRECTION', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP',
       'ACTUALTIME_ARR', 'ACTUALTIME_DEP', 'DAYSTAMP', 'ROUTEID'],
      dtype='object')

In [35]:
trips=trips[['DAYSTAMP','TRIPID', 'LINEID','ROUTEID','DIRECTION', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP',
       'ACTUALTIME_ARR', 'ACTUALTIME_DEP',]]

In [37]:
trips.head()

,DAYSTAMP,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,1517961600,6253783,68,80,1,87245,84600,87524.0,84600.0
1,1517961600,6262138,25B,271,2,30517,26460,32752.0,NaN
2,1517961600,6254942,45A,70,2,35512,32100,36329.0,32082.0
3,1517961600,6259460,25A,273,1,57261,54420,58463.0,54443.0
4,1517961600,6253175,14,15,1,85383,81600,84682.0,81608.0


In [38]:
trips.to_csv("trips_cleaned.csv")